In [24]:
# Copyright (c) 2024 Microsoft Corporation.
# Licensed under the MIT License.

## API Overview

This notebook provides a demonstration of how to interact with graphrag as a library using the API as opposed to the CLI. Note that graphrag's CLI actually connects to the library through this API for all operations. 

In [25]:
from pathlib import Path
from pprint import pprint

import pandas as pd

import graphrag.api as api
from graphrag.config.load_config import load_config
from graphrag.index.typing.pipeline_run_result import PipelineRunResult

In [ ]:
PROJECT_DIRECTORY = "<your project directory>"

## Prerequisite
As a prerequisite to all API operations, a `GraphRagConfig` object is required. It is the primary means to control the behavior of graphrag and can be instantiated from a `settings.yaml` configuration file.

Please refer to the [CLI docs](https://microsoft.github.io/graphrag/cli/#init) for more detailed information on how to generate the `settings.yaml` file.

### Generate a `GraphRagConfig` object

In [27]:
graphrag_config = load_config(Path(PROJECT_DIRECTORY))

## Indexing API

*Indexing* is the process of ingesting raw text data and constructing a knowledge graph. GraphRAG currently supports plaintext (`.txt`) and `.csv` file formats.

## Build an index

In [28]:
index_result: list[PipelineRunResult] = await api.build_index(config=graphrag_config)

# index_result is a list of workflows that make up the indexing pipeline that was run
for workflow_result in index_result:
    status = f"error\n{workflow_result.errors}" if workflow_result.errors else "success"
    print(f"Workflow Name: {workflow_result.workflow}\tStatus: {status}")

No report found for community: 21.0
No report found for community: 22.0
No report found for community: 19.0
No report found for community: 17.0
No report found for community: 18.0
No report found for community: 26.0
No report found for community: 27.0
No report found for community: 15.0
No report found for community: 20.0
No report found for community: 16.0
No report found for community: 30.0
No report found for community: 24.0
No report found for community: 23.0
No report found for community: 32.0
No report found for community: 28.0
No report found for community: 36.0
No report found for community: 25.0
No report found for community: 33.0
No report found for community: 29.0
No report found for community: 37.0
No report found for community: 38.0
No report found for community: 31.0
No report found for community: 39.0
No report found for community: 35.0
No report found for community: 34.0
No report found for community: 40.0
No report found for community: 41.0
No report found for communit

Workflow Name: create_base_text_units	Status: success
Workflow Name: create_final_documents	Status: success
Workflow Name: extract_graph	Status: success
Workflow Name: finalize_graph	Status: success
Workflow Name: extract_covariates	Status: success
Workflow Name: create_communities	Status: success
Workflow Name: create_final_text_units	Status: success
Workflow Name: create_community_reports	Status: error
[KeyError('community')]


## Query an index

To query an index, several index files must first be read into memory and passed to the query API. 

In [29]:
entities = pd.read_parquet(f"{PROJECT_DIRECTORY}/output/entities.parquet")
communities = pd.read_parquet(f"{PROJECT_DIRECTORY}/output/communities.parquet")
community_reports = pd.read_parquet(
    f"{PROJECT_DIRECTORY}/output/community_reports.parquet"
)

response, context = await api.global_search(
    config=graphrag_config,
    entities=entities,
    communities=communities,
    community_reports=community_reports,
    community_level=2,
    dynamic_community_selection=False,
    response_type="Multiple Paragraphs",
    query="Who is Scrooge and what are his main relationships?",
)

The response object is the official reponse from graphrag while the context object holds various metadata regarding the querying process used to obtain the final response.

In [30]:
print(response)

### Ebenezer Scrooge: Character Overview

Ebenezer Scrooge, often referred to simply as Scrooge, is the central figure in the community. Initially, he is depicted as a wealthy, miserly, and unfeeling man, known for his uncharitable nature, particularly during Christmas time. Scrooge lives alone in a gloomy suite of rooms that once belonged to his deceased business partner, Jacob Marley. His practical and unimaginative nature makes him a solitary figure, described as a 'covetous old sinner' [Data: Reports (0, 1, 7, 25, 36)].

### Key Relationships

#### Jacob Marley
Scrooge's most significant relationship is with Jacob Marley, his deceased business partner. Marley's ghost visits Scrooge to warn him of the impending visits from other spirits, setting the stage for Scrooge's transformative journey. Marley plays a crucial role in initiating Scrooge's transformation by predicting the visit of three spirits who will guide Scrooge through scenes of his past, present, and future [Data: Reports

Digging into the context a bit more provides users with extremely granular information such as what sources of data (down to the level of text chunks) were ultimately retrieved and used as part of the context sent to the LLM model).

In [31]:
pprint(context)  # noqa: T203

{'reports':     id                                              title  occurrence weight  \
0    1               Scrooge and the Spirits of Christmas           0.612903   
1    7                       Scrooge and Marley Community           0.580645   
2   10          Christmas Celebrations and Key Characters           0.354839   
3   37  Scrooge and Marley Business and Supernatural E...           0.290323   
4   25                 Scrooge and Bob Cratchit Community           0.258065   
5   36                 Scrooge and Jacob Marley Community           0.225806   
6   45          Project Gutenberg and its Legal Framework           0.193548   
7    2        Christmas Day Celebrations and Key Entities           0.161290   
8   33              Scrooge and the Phantom's Revelations           0.129032   
9   38  Project Gutenberg Literary Archive Foundation ...           0.129032   
10  12       Joe's Shop and the Deceased Man's Belongings           0.096774   
11   4           Fezziwig's 